In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

import deeplabcut

colors = [
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'blue','blue','blue',
'red','red','red',
'green','green','green',
'cyan','cyan','cyan',
'orange','orange','orange',
    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [2]:
def create_video_with_all_detections(config, videos, DLCscorername, destfolder=None):
    """
    Create a video labeled with all the detections stored in a '*_full.pickle' file.
    Parameters
    ----------
    config : str
        Absolute path to the config.yaml file
    videos : list of str
        A list of strings containing the full paths to videos for analysis or a path to the directory,
        where all the videos with same extension are stored.
    DLCscorername: str
        Name of network. E.g. 'DLC_resnet50_project_userMar23shuffle1_50000
    destfolder: string, optional
        Specifies the destination folder that was used for storing analysis data (default is the path of the video).
    """
    from deeplabcut.pose_estimation_tensorflow.lib.inferenceutils import (
        convertdetectiondict2listoflist,
    )
    import pickle, re
    from deeplabcut import auxiliaryfunctions
    from pathlib import Path
    from tqdm import trange
    from skimage.draw import circle, line_aa
    
    from deeplabcut.utils.video_processor import (
    VideoProcessorCV as vp)  # used to CreateVideo
    
    cfg = auxiliaryfunctions.read_config(config)

    out_array = []
    
    for video in videos:
        videofolder = os.path.splitext(video)[0]

        if destfolder is None:
            outputname = "{}_full.mp4".format(videofolder + DLCscorername)
            full_pickle = os.path.join(videofolder + DLCscorername + "_full.pickle")
        else:
            auxiliaryfunctions.attempttomakefolder(destfolder)
            outputname = os.path.join(
                destfolder, str(Path(video).stem) + DLCscorername + "_full.mp4"
            )
            full_pickle = os.path.join(
                destfolder, str(Path(video).stem) + DLCscorername + "_full.pickle"
            )

        if not (os.path.isfile(outputname)):
            print("Creating labeled video for ", str(Path(video).stem))
            with open(full_pickle, "rb") as file:
                data = pickle.load(file)

            header = data.pop("metadata")
            all_jointnames = header["all_joints_names"]

            numjoints = len(all_jointnames)
            bpts = range(numjoints)
            frame_names = list(data)
            frames = [int(re.findall(r"\d+", name)[0]) for name in frame_names]
            colorclass = plt.cm.ScalarMappable(cmap=cfg["colormap"])
            C = colorclass.to_rgba(np.linspace(0, 1, numjoints))
            colors = (C[:, :3] * 255).astype(np.uint8)
            print ("COLORSL ", colors)

            pcutoff = cfg["pcutoff"]
            dotsize = cfg["dotsize"]
            clip = vp(fname=video, sname=outputname, codec="mp4v")
            ny, nx = clip.height(), clip.width()

            print ("clip.nframes: ", clip.nframes)
            print ("frames: ", len(frames))
            print ("pcutoff: ", pcutoff)
            pcutoff = 0
            for n in trange(clip.nframes):
                frame = clip.load_frame()
                try:
                    ind = frames.index(n)
                    dets = convertdetectiondict2listoflist(data[frame_names[ind]], bpts)
                    
                    print (n, " len dets: ", len(dets))
                    print (dets[0])
                    for i, det in enumerate(dets):
                        color = colors[i]
                        for x, y, p, _ in det:
                            print (i, " x, y, p:", x, y, p)
                            if p > pcutoff:
                                rr, cc = circle(y, x, dotsize, shape=(ny, nx))
                                frame[rr, cc] = color
                        break
                except ValueError:  # No data stored for that particular frame
                    print(n, "no data")
                    pass
                
                try:
                    clip.save_frame(frame)
                except:
                    print(n, "frame writing error.")
                    pass
            clip.close()
        else:
            print("Detections already plotted, ", outputname)
            
def convert_detections2tracklets(
    config,
    videos,
    videotype="avi",
    shuffle=1,
    trainingsetindex=0,
    overwrite=False,
    destfolder=None,
    BPTS=None,
    iBPTS=None,
    PAF=None,
    printintermediate=False,
    inferencecfg=None,
    modelprefix="",
    track_method="box",
    edgewisecondition=True,
):
    """
    This should be called at the end of deeplabcut.analyze_videos for multianimal projects!
    Parameters
    ----------
    config : string
        Full path of the config.yaml file as a string.
    videos : list
        A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.
    videotype: string, optional
        Checks for the extension of the video in case the input to the video is a directory.\n Only videos with this extension are analyzed. The default is ``.avi``
    shuffle: int, optional
        An integer specifying the shuffle index of the training dataset used for training the network. The default is 1.
    trainingsetindex: int, optional
        Integer specifying which TrainingsetFraction to use. By default the first (note that TrainingFraction is a list in config.yaml).
    overwrite: bool, optional.
        Overwrite tracks file i.e. recompute tracks from full detections and overwrite.
    destfolder: string, optional
        Specifies the destination folder for analysis data (default is the path of the video). Note that for subsequent analysis this
        folder also needs to be passed.
    track_method: str, optional
        Method uses to track animals, either 'box' or 'skeleton'.
        By default, a constant velocity Kalman filter is used to track individual bounding boxes.
    BPTS: Default is None: all bodyparts are used.
        Pass list of indices if only certain bodyparts should be used (advanced).
    iBPTS: Default is None: all bodyparts are used.
        The inverse indices from BPTS.
        TODO: calculate from BPTS
    PAF: Default is None: all connections are used.
        Pass list of indices if only certain connections should be used (advanced).
    printintermediate: ## TODO
        Default is false.
    inferencecfg: Default is None.
        Configuaration file for inference (assembly of individuals). Ideally
        should be optained from cross validation (during evaluation). By default
        the parameters are loaded from inference_cfg.yaml, but these get_level_values
        can be overwritten.
    edgewisecondition: bool, default False.
        If true pairwise Euclidean distances of limbs (connections in PAF) will be
        estimated from the annotated data and used for excluding possible connections.
    Examples
    --------
    If you want to convert detections to tracklets:
    >>> deeplabcut.convert_detections2tracklets('/analysis/project/reaching-task/config.yaml',[]'/analysis/project/video1.mp4'], videotype='.mp4')
    --------
    """
    from deeplabcut.pose_estimation_tensorflow.lib import inferenceutils, trackingutils
    from deeplabcut.utils import auxfun_multianimal
    from easydict import EasyDict as edict
    import pickle

    if track_method not in ("box", "skeleton"):
        raise ValueError(
            "Invalid tracking method. Only `box` and `skeleton` are currently supported."
        )

    cfg = auxiliaryfunctions.read_config(config)
    trainFraction = cfg["TrainingFraction"][trainingsetindex]
    start_path = os.getcwd()  # record cwd to return to this directory in the end

    # TODO: addd cropping as in video analysis!
    # if cropping is not None:
    #    cfg['cropping']=True
    #    cfg['x1'],cfg['x2'],cfg['y1'],cfg['y2']=cropping
    #    print("Overwriting cropping parameters:", cropping)
    #    print("These are used for all videos, but won't be save to the cfg file.")

    modelfolder = os.path.join(
        cfg["project_path"],
        str(
            auxiliaryfunctions.GetModelFolder(
                trainFraction, shuffle, cfg, modelprefix=modelprefix
            )
        ),
    )
    path_test_config = Path(modelfolder) / "test" / "pose_cfg.yaml"
    try:
        dlc_cfg = load_config(str(path_test_config))
    except FileNotFoundError:
        raise FileNotFoundError(
            "It seems the model for shuffle %s and trainFraction %s does not exist."
            % (shuffle, trainFraction)
        )

    print ("path_test_config: ", path_test_config)
    if "multi-animal" not in dlc_cfg["dataset_type"]:
        raise ValueError("This function is only required for multianimal projects!")

    path_inference_config = Path(modelfolder) / "test" / "inference_cfg.yaml"
    if inferencecfg is None:  # then load or initialize
        inferencecfg = auxfun_multianimal.read_inferencecfg(path_inference_config, cfg)
    else:
        inferencecfg = edict(inferencecfg)
        auxfun_multianimal.check_inferencecfg_sanity(cfg, inferencecfg)

    if edgewisecondition:
        path_inferencebounds_config = (
            Path(modelfolder) / "test" / "inferencebounds.yaml"
        )
        try:
            inferenceboundscfg = auxiliaryfunctions.read_plainconfig(
                path_inferencebounds_config
            )
        except FileNotFoundError:
            print("Computing distances...")
            from deeplabcut.pose_estimation_tensorflow import calculatepafdistancebounds

            inferenceboundscfg = calculatepafdistancebounds(
                config, shuffle, trainingsetindex
            )
            auxiliaryfunctions.write_plainconfig(
                path_inferencebounds_config, inferenceboundscfg
            )

    # Check which snapshots are available and sort them by # iterations
    try:
        Snapshots = np.array(
            [
                fn.split(".")[0]
                for fn in os.listdir(os.path.join(modelfolder, "train"))
                if "index" in fn
            ]
        )
    except FileNotFoundError:
        raise FileNotFoundError(
            "Snapshots not found! It seems the dataset for shuffle %s has not been trained/does not exist.\n Please train it before using it to analyze videos.\n Use the function 'train_network' to train the network for shuffle %s."
            % (shuffle, shuffle)
        )

    if cfg["snapshotindex"] == "all":
        print(
            "Snapshotindex is set to 'all' in the config.yaml file. Running video analysis with all snapshots is very costly! Use the function 'evaluate_network' to choose the best the snapshot. For now, changing snapshot index to -1!"
        )
        snapshotindex = -1
    else:
        snapshotindex = cfg["snapshotindex"]

    print ("Snapshots: ", Snapshots)
    increasing_indices = np.argsort([int(m.split("-")[1]) for m in Snapshots])
    print ("increasing_indices: ", increasing_indices)
    Snapshots = Snapshots[increasing_indices]
    print ("modelfolder: ", modelfolder)
    print("Using %s" % Snapshots[snapshotindex], "for model", modelfolder)
    dlc_cfg["init_weights"] = os.path.join(
        modelfolder, "train", Snapshots[snapshotindex]
    )
    trainingsiterations = (dlc_cfg["init_weights"].split(os.sep)[-1]).split("-")[-1]

    # Name for scorer:
    DLCscorer, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(
        cfg,
        shuffle,
        trainFraction,
        trainingsiterations=trainingsiterations,
        modelprefix=modelprefix,
    )

    ##################################################
    # Looping over videos
    ##################################################
    Videos = auxiliaryfunctions.Getlistofvideos(videos, videotype)
    if len(Videos) > 0:
        for video in Videos:
            print("Processing... ", video)
            videofolder = str(Path(video).parents[0])
            if destfolder is None:
                destfolder = videofolder
            auxiliaryfunctions.attempttomakefolder(destfolder)
            vname = Path(video).stem
            dataname = os.path.join(videofolder, vname + DLCscorer + ".h5")
            data, metadata = auxfun_multianimal.LoadFullMultiAnimalData(dataname)
            method = "sk" if track_method == "skeleton" else "bx"
            trackname = dataname.split(".h5")[0] + f"_{method}.pickle"
            trackname = trackname.replace(videofolder, destfolder)
            if (
                os.path.isfile(trackname) and not overwrite
            ):  # TODO: check if metadata are identical (same parameters!)
                print("Tracklets already computed", trackname)
                print("Set overwrite = True to overwrite.")
            else:
                print("Analyzing", dataname)
                DLCscorer = metadata["data"]["Scorer"]
                dlc_cfg = metadata["data"]["DLC-model-config file"]
                nms_radius = data["metadata"]["nms radius"]
                minconfidence = data["metadata"]["minimal confidence"]
                print ("minconfidence: ", minconfidence)
                #minconfidence = 0.

                partaffinityfield_graph = data["metadata"]["PAFgraph"]
                all_joints = data["metadata"]["all_joints"]
                all_jointnames = data["metadata"]["all_joints_names"]

                if edgewisecondition:
                    upperbound = np.array(
                        [
                            float(
                                inferenceboundscfg[str(edge[0]) + "_" + str(edge[1])][
                                    "intra_max"
                                ]
                            )
                            for edge in partaffinityfield_graph
                        ]
                    )
                    lowerbound = np.array(
                        [
                            float(
                                inferenceboundscfg[str(edge[0]) + "_" + str(edge[1])][
                                    "intra_min"
                                ]
                            )
                            for edge in partaffinityfield_graph
                        ]
                    )
                    upperbound *= 1.25
                    lowerbound *= 0.5  # SLACK!
                else:
                    lowerbound = None
                    upperbound = None

                #lowerbound += 40
                
                if PAF is None:
                    PAF = np.arange(
                        len(partaffinityfield_graph)
                    )  # THIS CAN BE A SUBSET!

                partaffinityfield_graph = [partaffinityfield_graph[l] for l in PAF]
                linkingpartaffinityfield_graph = partaffinityfield_graph

                numjoints = len(all_jointnames)
                if BPTS is None and iBPTS is None:
                    # NOTE: this can be used if only a subset is relevant. I.e. [0,1] for only first and second joint!
                    BPTS = range(numjoints)
                    iBPTS = range(numjoints)  # the corresponding inverse!

                # TODO: adjust this for multi + unique bodyparts!
                # this is only for multianimal parts and uniquebodyparts as one (not one uniquebodyparts guy tracked etc. )
                bodypartlabels = sum([3 * [all_jointnames[bpt]] for bpt in BPTS], [])
                numentries = len(bodypartlabels)

                scorers = numentries * [DLCscorer]
                xylvalue = int(len(bodypartlabels) / 3) * ["x", "y", "likelihood"]
                pdindex = pd.MultiIndex.from_arrays(
                    np.vstack([scorers, bodypartlabels, xylvalue]),
                    names=["scorer", "bodyparts", "coords"],
                )

                imnames = [fn for fn in data if fn != "metadata"]

                if track_method == "box":
                    mot_tracker = trackingutils.Sort(inferencecfg)
                else:
                    mot_tracker = trackingutils.SORT(
                        numjoints,
                        inferencecfg["max_age"],
                        inferencecfg["min_hits"],
                        inferencecfg.get("oks_threshold", 0.5),
                    )
                    
                print ("inferencecfg: ", inferencecfg)
                print ("numjoints: ", numjoints)
                print ("BPTS: ", BPTS)
                print ("iBPTS: ", iBPTS)
                print ("PAF: ", PAF)
                print ("partaffinityfield_graph: ", partaffinityfield_graph)
                print ("linkingpartaffinityfield_graph: ", linkingpartaffinityfield_graph)
                
                #print (" data[imname]: ", data[imnames[0]])
                print ("lowerbound: ", lowerbound)
                print ("upperbound: ", upperbound)
                print ("printintermediate: ", printintermediate)
                
                Tracks = {}
                for index, imname in tqdm(enumerate(imnames)):
                    animals = inferenceutils.assemble_individuals(
                        inferencecfg,
                        data[imname],
                        numjoints,
                        BPTS,
                        iBPTS,
                        PAF,
                        partaffinityfield_graph,
                        linkingpartaffinityfield_graph,
                        lowerbound,
                        upperbound,
                        printintermediate,
                    )
                    print ("Animals: ", animals)
                    if track_method == "box":
                        # get corresponding bounding boxes!
                        bb = inferenceutils.individual2boundingbox(
                            inferencecfg, animals, 0
                        )  # TODO: get cropping parameters and utilize!
                        trackers = mot_tracker.update(bb)
                    else:
                        temp = [arr.reshape((-1, 3))[:, :2] for arr in animals]
                        trackers = mot_tracker.track(temp)
                    trackingutils.fill_tracklets(Tracks, trackers, animals, imname)

                    # Test whether the unique bodyparts have been assembled
                    # TODO Perhaps easier to check whether links were defined in the PAF graph?
                    inds_unique = [
                        all_jointnames.index(bp) for bp in cfg["uniquebodyparts"]
                    ]
                    if not any(
                        np.isfinite(a.reshape((-1, 3))[inds_unique]).all()
                        for a in animals
                    ):
                        single = np.full((numjoints, 3), np.nan)
                        single_dets = inferenceutils.convertdetectiondict2listoflist(
                            data[imname], inds_unique
                        )
                        for ind, dets in zip(inds_unique, single_dets):
                            if len(dets) == 1:
                                single[ind] = dets[0][:3]
                            elif len(dets) > 1:
                                best = sorted(dets, key=lambda x: x[2], reverse=True)[0]
                                single[ind] = best[:3]
                        # Find an unused tracklet ID for the 'unique' bodyparts
                        tracklet_id = 0
                        while True:
                            if tracklet_id not in Tracks:
                                break
                            tracklet_id += 1
                        Tracks[tracklet_id] = {}
                        Tracks[tracklet_id][imname] = single.flatten()

                Tracks["header"] = pdindex
                with open(trackname, "wb") as f:
                    # Pickle the 'labeled-data' dictionary using the highest protocol available.
                    pickle.dump(Tracks, f, pickle.HIGHEST_PROTOCOL)

        os.chdir(str(start_path))

        print("The tracklets were created. Now you can 'refine_tracklets'.")
        # print("If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!")
    else:
        print("No video(s) found. Please check your path!")

        
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]
    print ("data labels: ", labels)
    print ("column vals: ", data[2])

    # load values
    data_array = np.array(data[3:])
    print ("# of datapoints (x,y,likelihood): ", data_array.shape)

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    # zero out low quality DLC values
    for idx in range(1,len(labels)-1,3):
        #print ("idx: ", idx)
        #print (data_array[1:,idx:idx+3])
        #print (data_array[1:,idx:idx+3].shape)

        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)
        print (idx, "TEMP: ", temp.shape)
        #temp.replace("''",'0')
        

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan


In [3]:
# LOAD LABELS FROM .BX.H5 FILE"
fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_filtered.h5'
#fname = '/media/cat/4TBSSD/Downloads/2020-3-8_12_08_57_943006_compressed_3min_sameDLC_resnet50_cat3Jun12shuffle1_50000_bx.h5'
hf = h5py.File(fname, 'r')
print (hf.keys())
data = hf.get('df_with_missing')
print (data.keys())
table = data.get('table')
print (len(table))
# itable = data.get('_i_table').get('index')
# bbounds = itable.get('zbounds')
print (table[1000])

<KeysViewHDF5 ['df_with_missing']>
<KeysViewHDF5 ['_i_table', 'table']>
16572
(1000, [3.71528000e+02, 7.39845000e+02, 8.77170026e-01, 3.56354000e+02, 7.16225000e+02, 9.99989986e-01, 3.34104000e+02, 7.39550000e+02, 6.86460018e-01, 3.31267000e+02, 7.01443000e+02, 9.99710023e-01, 3.10870000e+02, 7.31119000e+02, 9.99909997e-01, 3.26988000e+02, 7.21303000e+02, 9.96490002e-01, 2.98469000e+02, 7.07517000e+02, 9.99830008e-01, 2.64893000e+02, 6.83687000e+02, 9.98969972e-01,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan,            nan, 3.95512000e+02, 3.08761000e+02, 9.94949996e-01,            nan,            nan,            nan, 3.93853000e+02, 3.37104000e+02, 9.99390006e-01, 3.47638000e+02, 3.32733000e+02, 9.96110022e-01, 3.76865000e+02, 3.55022000e+02, 9.99880016e-01, 3.

In [4]:
# LOAD PICLED TRACKLETS 
fname1 = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'
#fname2 = '/media/cat/14TB/insync_cm5636/march_2/video/dlc_2.26b_results/march_2_redo-cat-2020-06-17/videos/new/2020-3-8_12_08_57_943006_compressed_3min_sameDLC_resnet50_march_2_redoJun17shuffle1_50000_bx.pickle'

data_pickle = np.load(fname1,allow_pickle=True)
print (data_pickle.keys())
#print (data_pickle['data']['cropping_parameters'])


dict_keys(['frame00000', 'frame00001', 'frame00002', 'frame00003', 'frame00004', 'frame00005', 'frame00006', 'frame00007', 'frame00008', 'frame00009', 'frame00010', 'frame00011', 'frame00012', 'frame00013', 'frame00014', 'frame00015', 'frame00016', 'frame00017', 'frame00018', 'frame00019', 'frame00020', 'frame00021', 'frame00022', 'frame00023', 'frame00024', 'frame00025', 'frame00026', 'frame00027', 'frame00028', 'frame00029', 'frame00030', 'frame00031', 'frame00032', 'frame00033', 'frame00034', 'frame00035', 'frame00036', 'frame00037', 'frame00038', 'frame00039', 'frame00040', 'frame00041', 'frame00042', 'frame00043', 'frame00044', 'frame00045', 'frame00046', 'frame00047', 'frame00048', 'frame00049', 'frame00050', 'frame00051', 'frame00052', 'frame00053', 'frame00054', 'frame00055', 'frame00056', 'frame00057', 'frame00058', 'frame00059', 'frame00060', 'frame00061', 'frame00062', 'frame00063', 'frame00064', 'frame00065', 'frame00066', 'frame00067', 'frame00068', 'frame00069', 'frame000

In [6]:
# CONVERT H5 DATA TO OLD CVS FORMAAT
path_to_video = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
deeplabcut.analyze_videos_converth5_to_csv(path_to_video, '.avi')

Found output file for scorer: DLC_resnet50_madeline_july2Jul2shuffle1_100000_bx
Converting to csv...
Found output file for scorer: DLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_filtered
Converting to csv...
All pose files were converted.


In [9]:
# if False:
#     print (data_pickle['frame00000'].keys())
#     # for k in range(14):
#     #     print (np.array(data_pickle['frame0000']['coordinates'][0][k]).shape)
#     #     print (data_pickle['frame0000']['confidence'][0].squeeze().shape)

#     print (data_pickle['frame00000']['costs'].keys())
#     print (data_pickle['frame00000']['costs'][0].keys())
#     for k in range(len(data_pickle['frame00000']['costs'])):
#         print (data_pickle['frame00000']['costs'][k]['m1'].shape)
#         print (data_pickle['frame00000']['costs'][k]['distance'].shape)
#         print ("")

In [14]:
# CREATE A VIDEO WITH ALL THE DETECTIONS
            
config = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/config.yaml'
videos = ['/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi']
#DLCscorername = 'DLC_resnet50_march_2_redoJun17shuffle1_50000'
DLCscorername = 'DLC_resnet50_madeline_july2Jul2shuffle1_100000'
destfolder = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
create_video_with_all_detections(config, videos, DLCscorername, destfolder)

    

/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/  already exists!
Detections already plotted,  /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.mp4


In [16]:
# covnert detections to tracklets with some thresholding
                
#config = '/media/cat/14TB/insync_cm5636/march_2/video/dlc_2.26b_results/march_2_redo-cat-2020-06-17/config.yaml'
#videos = ['/media/cat/14TB/insync_cm5636/march_2/video/dlc_2.26b_results/march_2_redo-cat-2020-06-17/videos/new/2020-3-8_12_08_57_943006_compressed_3min_same.avi']
#DLCscorername = 'DLC_resnet50_march_2_redoJun17shuffle1_50000'
#destfolder = '/media/cat/14TB/insync_cm5636/march_2/video/dlc_2.26b_results/march_2_redo-cat-2020-06-17/videos/new/'   

from deeplabcut.pose_estimation_tensorflow.config import load_config
#from deeplabcut.pose_estimation_tensorflow.nnet import predict
import pickle, re
from deeplabcut import auxiliaryfunctions
from pathlib import Path
from tqdm import trange
from skimage.draw import circle, line_aa
from tqdm import tqdm

convert_detections2tracklets(config,
    videos,
    videotype="avi",
    shuffle=1,
    trainingsetindex=0,
    overwrite=False,
    destfolder=None,
    BPTS=None,
    iBPTS=None,
    PAF=None,
    printintermediate=False,
    inferencecfg=None,
    modelprefix="",
    track_method="box",
    edgewisecondition=True)



path_test_config:  /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1/test/pose_cfg.yaml
Snapshots:  ['snapshot-90000' 'snapshot-80000' 'snapshot-95000' 'snapshot-100000'
 'snapshot-85000']
increasing_indices:  [1 4 0 2 3]
modelfolder:  /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1
Using snapshot-100000 for model /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1
Processing...  /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi
/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168  already exists!
Tracklets already computed /media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_1

In [5]:
csv_fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippetDLC_resnet50_madeline_july2Jul2shuffle1_100000_bx_filtered.csv'
data_all = load_csv(csv_fname)

data labels:  ['individuals', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adu

In [6]:
# CONVERT THE TABLE VALUES TO X,Y and LIKELIHOOD LOCATIONS;
traces = np.array(data_all[0])
#print (traces[0].shape)
tracesx = []
tracesy = []
likelihoods = []

for k in range(0,len(traces),1):
    tracesx.append(traces[k][:,0])
    tracesy.append(traces[k][:,1])
    likelihoods.append(traces[k][:,2])

likelihoods = np.array(likelihoods)
tracesx = np.array(tracesx)
tracesy = np.array(tracesy)
print ("tracex: ", tracesx.shape)
print ("tracey: ", tracesy.shape)
print ("likelihood: ", likelihoods.shape)


import matplotlib.cm as cm
cmap = cm.get_cmap('viridis',likelihoods.shape[0])

# OPTIONAL VISUALIZE FEATURES AS SCATTER PLOTS
if False:
    for feature in range(len(traces))[:1]:
        print (likelihoods.shape, feature)
        clrs = cmap(np.arange(likelihoods.shape[1]))*likelihoods[feature][:,None]
        ax=plt.subplot(7,8,feature+1)
        plt.scatter(tracesx[feature], tracesy[feature], color=clrs)

        plt.ylim(0,1024)
        plt.xlim(0,1280)
        plt.xticks([])
        plt.yticks([])
    plt.show()
   

tracex:  (56, 16572)
tracey:  (56, 16572)
likelihood:  (56, 16572)


In [7]:
# OPTIONAL VISUALIZE FEATURES AS SCATTER PLOTS
animals = []
print (traces.shape)
for k in range(0, len(traces),14):
    x=np.nanmean(traces[k:k+14,:,0], 0)
    y=np.nanmean(traces[k:k+14,:,1], 0)
    l=np.nanmean(traces[k:k+14,:,2], 0)
    animals.append([x,y,l])
    #y=traces[k:k+14,:,1].mean(0)
    #z=traces[k:k+14,:,2].mean(0)

cmaps = ['viridis','plasma','magma','inferno']
animal_id = ['female','male','pup1','pup2']
box = [[75,800],[100,900]]
for k in range(len(animals)):
#for k in [0]:
    ax=plt.subplot(2,2,k+1)
    plt.scatter(animals[k][0], animals[k][1], c=np.arange(animals[k][0].shape[0]), cmap='viridis')
    plt.title("Animal: "+animal_id[k])
    plt.ylim(0,1024)
    plt.xlim(0,1280)

    plt.plot([80,80],[25,800],c='black')
    plt.plot([80,1150],[800,800],c='black')
    plt.plot([1150,1150],[800,25],c='black')
    plt.plot([80,1150],[25,25],c='black')
plt.plot([80,1150],[25,25],c='black', label='cage')
plt.legend(fontsize=15)

plt.show()

(56, 16572, 3)


In [8]:
# LOAD VIDE TO ANNOTATE
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"
frame_rate = 25


In [78]:
# video writing step
#video_out.release()
#out.release()
#original_vid.release()
#cv2.destroyAllWindows()

# Robust outlier detection;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

       
original_vid = cv2.VideoCapture(video_name)
#cap = cv2.VideoCapture('chaplin.mp4')

ctr=0
# figure out index of starting frame and move video to location
frame_no1 = 0 #56*60*frame_rate
original_vid.set(1,frame_no1)

# initialize video writer for file to be saved
out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/vids_out/'


# Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
n_sec = 180
clrs = ['blue','red','yellow','green']

# 
#for k in range(tracesx.shape[1]):    
#for k in range(tracesx.shape[1])[:500]:    
ctr=0
start = 7*60*25
end = start + 120*25

fname_out = out_dir+'video_labeled_'+str(start)+"_"+str(end)+'.avi'
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
# fourCc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (1280,1024), True)


start_frame_number = start
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)

for k in range(start,end,1):    
    if k %100==0:
        print (k)
    
    #while True:
    ret, frame = original_vid.read()
    if ret==False:
        break

#         if ctr==k:
#             ctr+=1
#             break
#         ctr+=1
            
    #frame_cropped = np.flipud(frame.copy())#[:,150:1174][::2,::2]
    frame_cropped = frame.copy()  #[:,150:1174][::2,::2]

    # PLOT EVERY FEATURE
    threshold = 0.95
    if False:
        for p in range(tracesx.shape[0]):
            l = likelihoods[p,k]
            if l < threshold:
                continue

            x = np.int32(tracesx[p,k])

            y = np.int32(tracesy[p,k])


            # plot individual features
            frame_cropped[y-5:y+5,x-5:x+5]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    
    # PLOT MEAN FEATURE
    for p in range(0, tracesx.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]

        traces_local = tracesx[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        x = np.int32(np.nanmedian(traces_local))
        
        traces_local = tracesy[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        y = np.int32(np.nanmedian(traces_local))
        
        # plot large bump
        frame_cropped[y-15:y+15,x-15:x+15]= (np.float32(
            matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    

    #cv2.imshow('frame',frame_cropped)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #   break

    video_out.write(frame_cropped)
    #out.write(frame_cropped)
    
video_out.release()
original_vid.release()
cv2.destroyAllWindows()

#break
    

10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400


In [134]:
import pandas

df = pandas.read_excel('/media/cat/1TB/insync_cm5635/march_2/video/completed/2020-3-15_03:51:56:415333_compressed/2020-3-15_03_51_56_415333.xlsx')

# print the column names
print (df.columns)

# get the values for a given column
times_ = df[df.columns[0]].values
time_array = []
for t in times_:
    time_array.append((t.hour * 60 + t.minute) * 60 + t.second)

male = df[df.columns[1]].values
female = df[df.columns[2]].values
pup1 = df[df.columns[3]].values
pup2 = df[df.columns[4]].values

# 
idx = np.where(male=='A-C')[0]
male[idx]=13
idx = np.where(male=='A-B')[0]
male[idx]=13

idx = np.where(female=='B-D')[0]
female[idx]=13
idx = np.where(female=='B-C')[0]
female[idx]=13

idx = np.where(pup1=='B-C')[0]
pup1[idx]=13
idx = np.where(pup1=='C-D')[0]
pup1[idx]=13
idx = np.where(pup1=='A-C')[0]
pup1[idx]=13

idx = np.where(pup2=='A-C')[0]
pup2[idx]=13
idx = np.where(pup2=='A-B')[0]
pup2[idx]=13
idx = np.where(pup2=='C-D')[0]
pup2[idx]=13
idx = np.where(pup2=='A-D')[0]
pup2[idx]=13
idx = np.where(pup2=='B-D')[0]
pup2[idx]=13
#
plt.plot(male, c='blue', label="male")
plt.plot(female, c='red', label="female")
plt.plot(pup1, c='green', label="pup1")
plt.plot(pup2, c='cyan', label="pup2")
plt.xlabel("Time (sec)", fontsize=20)
plt.ylabel("States",fontsize=20)
plt.legend(fontsize=20)

plt.scatter(start, start*0+13, c='black')

labels = [
    "1 Scratching walls",
    "2 Digging beddings",
    "3 Playing (scratching) with blocks or hut",
    "4 Self grooming",
    "5 Sleeping",
    "6 Walking",
    "7 Chasing",
    "8 Being Chased",
    "9 Eating",
    "10 Drinking/ at spout",
    "11 Occluded",
    "12",
    "13 Social Interaction"
]
plt.yticks(np.arange(1,14,1),labels)



plt.show()


Index(['Unnamed: 0', 'A-Male adult', 'B-Female adult', 'C-fluffy pup',
       'D-shaved pup', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [30]:
# DLC csv labeling frame; OLD METHOD USING .CSV FILES
fname = '/media/cat/14TB/insync_cm5636/march_2/video/dlc_2.26b_results/march_2_redo-cat-2020-06-17/videos/new/2020-3-8_12_08_57_943006_compressed_3min_sameDLC_resnet50_march_2_redoJun17shuffle1_50000_bx_filtered.csv'

traces, labels, traces_nan = load_csv(fname)




data labels:  ['individuals', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_female', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adult_male', 'adu

In [78]:
# plot time course of video vs. other data 
fname_vid = '/media/cat/1TB/insync_cm5635/march_2/video/completed/dan_USV_triggered/2020-3-15_03_51_56_415333/2020-3-15_03:51:56:415333_compressed.avi'
original_vid.release()
original_vid = cv2.VideoCapture(video_name)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

ctr=0
motion = []
ret, old_frame = original_vid.read()
while True:
    if ctr%1000==0:
        print (ctr)

    ret, frame = original_vid.read()
    
    #motion.append([np.sum(old_frame[:,:,0]-frame[:,:,0]),
    #              np.sum(old_frame[:,:,1]-frame[:,:,1]),
    #              np.sum(old_frame[:,:,2]-frame[:,:,2])])
    motion.append(np.sum(old_frame[:,:,1]-frame[:,:,1]))
    
    if ret==False:
        break
    #if ctr>1000:
    #    break
    ctr+=1
    old_frame = frame
        
motion = np.vstack(motion)
print (motion)
print (motion.shape)
fig=plt.figure()
plt.plot(motion,c='green')
# plt.plot(motion[:,1],c='green')
# plt.plot(motion[:,2],c='blue')
plt.show()


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000


TypeError: 'NoneType' object is not subscriptable

In [81]:
print (np.array(motion).shape)

np.save('/media/cat/1TB/insync_cm5635/march_2/video/completed/2020-3-15_03:51:56:415333_compressed/motion.npy', motion)

(89987,)


In [84]:
fs=25

f, Pxx_den = signal.periodogram(np.float32(motion).squeeze(), fs)
print (f)
print (f.shape)
print (Pxx_den.shape)
plt.semilogy(f, Pxx_den)
#plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.title("Video power spectrum")
plt.show()

[0.00000000e+00 2.77817907e-04 5.55635814e-04 ... 1.24993055e+01
 1.24995833e+01 1.24998611e+01]
(44994,)
(44994,)


In [71]:
# filter data


(100000,)


In [77]:
import skvideo.io  
fname_vid = '/media/cat/1TB/insync_cm5635/march_2/video/completed/dan_USV_triggered/2020-3-15_03_51_56_415333/2020-3-15_03:51:56:415333_compressed.avi'
videodata = skvideo.io.vread(fname_vid)  
print(videodata.shape)

MemoryError: Unable to allocate 330. GiB for an array with shape (89988, 1024, 1280, 3) and data type uint8